# Data Prep

In [1]:
from utils.data_reader import read_data
from random import shuffle
from math import ceil

def segment(string, chunk_size):
    '''
    Segments `string` into a list of strings of length `chunk_size`
    '''
    return [string[i:i + chunk_size] for i in range(0, len(string), chunk_size)]

def prep_data(authors, test_pct=0, chunk_size=0):
    '''
    Prep data to be used in models.
    
    `authors` is a list of authors to prep data for.
    
    `test_pct` is a number between 0 and 1 indicating how much of the
    additional poetry per author should be designated as the test data.
    The Iliad will always be in the test data.
    
    `chunk_size` is an integer indicating how many characters to segment
    each piece into. 0 indicates no segmenting
    
    Returns a dict of the form:
    {
        'train': {
            author_name: [text, text, ...],
            author_name: [text, text, ...],
            ...
        }
        'test': [
            (text, author_name),
            (text, author_name),
            ...
        ]
    }
    '''
    data = {
        'train': {},
        'test': []
    }
    raw_data = read_data(authors)
    
    # segment data if chunk_size > 0
    if chunk_size > 0:
        for auth in raw_data:
            raw_data[auth]['iliad'] = segment(raw_data[auth]['iliad'], chunk_size)
            raw_data[auth]['poetry'] = segment('\n'.join(raw_data[auth]['poetry']), chunk_size)
    
    # iliad always gets added to test data
    for auth in raw_data:
        iliad = raw_data[auth]['iliad']
        if type(iliad) == list:
            data['test'].extend([(text, auth) for text in iliad])
        elif type(iliad) == str:
            data['test'].append((iliad, auth))
    
    # split data into training and test
    for auth in raw_data:
        auth_data = raw_data[auth]['poetry']
        # shuffle so different texts are sorted into training and test (optional)
        shuffle(auth_data)
        
        idx = ceil(len(auth_data) * test_pct)
        test_set = auth_data[:idx]
        training_set = auth_data[idx:]
        
        data['train'][auth] = training_set
        data['test'].extend([(text, auth) for text in test_set])
        
    return data 

# Evaluating a Model

In [2]:
from collections import defaultdict
from utils.data_reader import read_data

def calculate_prf(matrix):
    '''
    Calculates precision, recall, and F1 score for the given
    confusion matrix. MUST be a simple 2x2 confusion matrix.
    
    Handles division by zero cases based on the following
    methodology: https://github.com/dice-group/gerbil/wiki/Precision,-Recall-and-F1-measure
    
    returns (precision, recall, F1 score) tuple
    '''
    if matrix['tp'] == 0:
        if matrix['fp'] == 0 and matrix['fn'] == 0:
            return 1.0, 1.0, 1.0
        if matrix['fp'] == 0 or matrix['fn'] == 0:
            return 0, 0, 0
    
    precision = matrix['tp'] / (matrix['tp'] + matrix['fp'])
    recall = matrix['tp'] / (matrix['tp'] + matrix['fn'])
    f1 = (2 * precision * recall) / (precision + recall)
    
    return precision, recall, f1

def score_model(model, data):
    '''
    Function to test the model on the given data.
    
    `data` should be in the form of the output of 
    the `prep_data` function.
    
    return (micro_f1, macro_f1, confusion_matrix) tuple
    '''
    training_data = data['train']
    test_data = data['test']

    model.train(training_data)
    
    # confusion matrix to store results used for calculating precision and recall
    # confusion_matrix[gold_label][model_label] = number of occurrences
    confusion_matrix = defaultdict(lambda: defaultdict(lambda: 0))
    
    for text, gold_label in test_data:
        model_label = model.identify(text)[0][1]        
        confusion_matrix[gold_label][model_label] += 1
    
    # compute individual class matrices
    class_matrix = {}
    num_results = sum([sum(confusion_matrix[auth].values()) for auth in confusion_matrix])
    
    for auth in confusion_matrix:
        class_matrix[auth] = {
            'tp': confusion_matrix[auth][auth],
            'fp': sum([confusion_matrix[i][auth] for i in confusion_matrix if i != auth]),
            'fn': sum(confusion_matrix[auth].values()) - confusion_matrix[auth][auth]
        }
        class_matrix[auth]['tn'] = num_results - sum(class_matrix[auth].values())
        
    # compute a pooled matrix
    pooled_matrix = {
        'tp': sum([class_matrix[auth]['tp'] for auth in class_matrix]),
        'fp': sum([class_matrix[auth]['fp'] for auth in class_matrix]),
        'tn': sum([class_matrix[auth]['tn'] for auth in class_matrix]),
        'fn': sum([class_matrix[auth]['fn'] for auth in class_matrix])
    }
    
    # micro precision, recall, and F1
    micro_precision, micro_recall, micro_f1 = calculate_prf(pooled_matrix)

    # macro precision, recall, and F1
    class_precision = {}
    class_recall = {}
    
    for auth in class_matrix:
        p, r, _ = calculate_prf(class_matrix[auth])
        class_precision[auth] = p
        class_recall[auth] = r
    
    macro_precision = sum(class_precision.values()) / len(class_precision)
    macro_recall = sum(class_recall.values()) / len(class_recall)
    macro_f1 = (2 * macro_precision * macro_recall) / (macro_precision + macro_recall)
    
    # return F1 scores and confusion matrix
    return micro_f1, macro_f1, confusion_matrix

In [3]:
# NOTE: add Cowper once the data is done
authors = ['Pope', 'Dryden', 'Chapman']

def test_model(model):
    '''
    Thoroughly test the given model.
    
    Model is evaluated on different chunk sizes, different
    training/test data amounts, and with different
    groupings of authors.
    
    Return a dictionary containing all of the results
    '''
    results = {}
    
    # do tests on several different chunk sizes
    # (add to the list of chunk sizes you want -- it makes things take very long!!)
    for chunk_size in [0]:
        # first perform a test on all authors
        results[f'{chunk_size}|all'] = score_model(model, prep_data(authors, chunk_size=chunk_size))

        # then do every pair-wise comparison
        for i in range(len(authors)):
            for j in range(i + 1, len(authors)):
                auth_1 = authors[i]
                auth_2 = authors[j]
                results[f'{chunk_size}|{auth_1} + {auth_2}'] = score_model(model, prep_data([auth_1, auth_2], chunk_size=chunk_size))
            
    return results

In [4]:
from models.compression_model import CompressionModel

test_model(CompressionModel())

{'0|all': (1.0,
  1.0,
  defaultdict(<function __main__.score_model.<locals>.<lambda>()>,
              {'Chapman': defaultdict(<function __main__.score_model.<locals>.<lambda>.<locals>.<lambda>()>,
                           {'Chapman': 1, 'Dryden': 0, 'Pope': 0}),
               'Dryden': defaultdict(<function __main__.score_model.<locals>.<lambda>.<locals>.<lambda>()>,
                           {'Dryden': 1, 'Chapman': 0, 'Pope': 0}),
               'Pope': defaultdict(<function __main__.score_model.<locals>.<lambda>.<locals>.<lambda>()>,
                           {'Pope': 1, 'Chapman': 0, 'Dryden': 0})})),
 '0|Pope + Dryden': (1.0,
  1.0,
  defaultdict(<function __main__.score_model.<locals>.<lambda>()>,
              {'Dryden': defaultdict(<function __main__.score_model.<locals>.<lambda>.<locals>.<lambda>()>,
                           {'Dryden': 1, 'Pope': 0}),
               'Pope': defaultdict(<function __main__.score_model.<locals>.<lambda>.<locals>.<lambda>()>,
            